In [ ]:
import json
import pandas
from concurrent.futures import ThreadPoolExecutor

In [ ]:
pandas.set_option('display.max_colwidth', None)

In [ ]:
# Chemin vers le fichier JSONL
filename = '/home/onyxia/work/openfoodfacts-products.jsonl'

In [ ]:
def pretty_print_first_lines(filename, num_lines=3):
    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            if i >= num_lines:
                break

            # Charger la ligne JSON
            data = json.loads(line)

            # Afficher le dictionnaire formaté
            print(json.dumps(data, indent=4))

In [ ]:
# Afficher les 3 premières lignes de manière jolie
pretty_print_first_lines(filename)

In [ ]:
def extract_lines_with_code(filename, code_value):
    
    lines_with_code = []

    with open(filename, 'r') as file:
        for line in file:
            # Charger la ligne JSON
            data = json.loads(line)

            # Vérifier si le champ "code" a la valeur spécifiée
            if data.get('code') == code_value:
                lines_with_code.append(data)

    return lines_with_code

In [ ]:
# Valeur du champ "code" à rechercher
code_value = "3272770005708"

# Extraction des lignes avec le champ "code" égal à la valeur spécifiée
lines_with_code = extract_lines_with_code(
    filename, 
    code_value
)

# Conversion en DataFrame pandas
df = pandas.DataFrame(lines_with_code)

# Affichage du DataFrame
print(df)

In [ ]:
df["ingredients"]

In [ ]:
df["ingredients"].to_string(index=False)

In [ ]:
pretty_json_str = json.dumps(df["ingredients"].to_string(index=False), indent=4)
print(pretty_json_str)

In [ ]:
filtered_data = {key: value for key, value in data.items() if key in ['code', 'ingredients']}

# Afficher de manière jolie le contenu du dictionnaire filtré
pretty_json_str = json.dumps(filtered_data, indent=4)
print(pretty_json_str)

In [ ]:
def pretty_print_json_column(df, column_name):
    # Convertir la liste d'objets JSON en une seule chaîne JSON
    json_str = df[column_name].apply(json.dumps)

    # Transformer les données JSON en DataFrame
    json_df = pandas.json_normalize(json_str.apply(json.loads))

    # Afficher de manière jolie le contenu du DataFrame JSON
    print(json_df.to_string(index=False))

In [ ]:
pretty_print_json_column(df, 'ingredients')

In [ ]:
def find_line_with_code(filename, code_value):
    with open(filename, 'r') as file:
        for line in file:
            # Charger la ligne JSON
            data = json.loads(line)

            # Vérifier si le champ "code" a la valeur spécifiée
            if data.get('code') == code_value:
                return data  # Retourner la ligne trouvée
            elif data.get('code') > code_value:
                break  # Arrêter la recherche si le code actuel dépasse la valeur recherchée

    return None  # Retourner None si aucune ligne avec le code spécifié n'est trouvée

In [ ]:
# Valeur du champ "code" à rechercher
code_value = "3272770005708"

# Récupération de la ligne avec le champ "code" égal à la valeur spécifiée
line_with_code = find_line_with_code(filename, code_value)

if line_with_code:
    # Affichage de la ligne trouvée
    print(line_with_code)
else:
    print("Aucune ligne trouvée avec le code spécifié.")

In [ ]:
def extract_lines_with_code_starting_with(filename, code_prefix):
    lines_with_code = []

    with open(filename, 'r') as file:
        for line in file:
            # Charger la ligne JSON
            data = json.loads(line)

            # Vérifier si le champ "code" commence par la valeur spécifiée
            if data.get('code', '').startswith(code_prefix):
                lines_with_code.append(data)

    return lines_with_code

In [ ]:
# Préfixe du champ "code" à rechercher
code_prefix = "300"

# Extraction des lignes avec le champ "code" commençant par le préfixe spécifié
lines_with_code = extract_lines_with_code_starting_with(
    filename, 
    code_prefix
)

# Conversion en DataFrame pandas
df = pandas.DataFrame(lines_with_code)

# Affichage du DataFrame
print(df)

In [24]:
def extract_lines_with_code_range(filename, start_code, end_code):
    lines_with_code = []

    with open(filename, 'r') as file:
        for line in file:
            # Charger la ligne JSON
            data = json.loads(line)

            # Récupérer les trois premiers caractères du champ "code"
            code_prefix = str(data.get('code', ''))[:3]

            # Vérifier si les trois premiers caractères du champ "code" sont numériques et compris entre start_code_prefix et end_code_prefix
            if code_prefix.isdigit() :
                if start_code_prefix <= int(code_prefix, 10) <= end_code_prefix:
                    lines_with_code.append(data)

In [ ]:
# Plage de valeurs des trois premiers caractères du champ "code"
start_code_prefix = 300
end_code_prefix = 379

# Extraction des lignes avec les trois premiers caractères du champ "code" compris entre start_code et end_code
lines_with_code_range = extract_lines_with_code_range(
    filename, 
    start_code_prefix, 
    end_code_prefix
)

# Conversion en DataFrame pandas
df = pandas.DataFrame(lines_with_code_range)

In [ ]:
df["ingredients"]

In [ ]:
df["ingredients"].to_string(index=False)

In [ ]:
def find_lines_with_code_range_multi_threads(filename, start_code, end_code):
    results = {}

    def search_code_range(start_code, end_code):
        with open(filename, 'r') as file:
            for line in file:
                # Charger la ligne JSON
                data = json.loads(line)

                # Récupérer la valeur du champ "code"
                code_value = data.get('code', '')

                # Vérifier si le champ "code" commence par une valeur comprise entre start_code et end_code
                if code_value.startswith(tuple(str(i) for i in range(start_code, end_code + 1))):
                    results[code_value] = data

    # Création d'un ThreadPoolExecutor avec 5 threads
    with ThreadPoolExecutor(max_workers=20) as executor:
        # Lancer les recherches de code sur chaque thread
        executor.submit(search_code_range, start_code, end_code)

    return results

In [ ]:
# Plage de valeurs de code à rechercher
start_code = 300
end_code = 379

# Recherche des lignes avec les codes spécifiés
lines_with_code_range = find_lines_with_code_range_multi_threads(filename, start_code, end_code)

# Affichage des lignes trouvées
for code_value, line in lines_with_code_range.items():
    print(f"Ligne avec le code {code_value}: {line}")